El paquete de _Python_ [scikit-learn](http://scikit-learn.org) (_sklearn_ en lo que sigue) proporciona un marco de trabajo para el aprendizaje automático.

# Aprendizaje supervisado

Para ilustrar el concepto de aprendizaje supervisado vamos a usar el conjunto de datos [_Car Evaluation_](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation) del repositorio [UCI](http://archive.ics.uci.edu/ml/). Este conjunto de datos contiene información acerca de la idoneidad de una serie de coches, en función de los siguientes atributos:
* _buying_: precio de compra. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _maint_: coste de mantenimiento. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _doors_: número de puertas. Posibles valores: `2`, `3`, `4`, `5more`.
* _persons_: número de asientos. Posibles valores: `2`, `4`, `more`.
* _lug\_boot_: tamaño del maletero. Posibles valores: `small`, `med`, `big`.
* _safety_: nivel de seguridad estimada. Posibles valores: `low`, `med`, `high`.

La idoneidad de cada coche se indica mediante el atributo _acceptability_, que los clasifica como `unacc`, `acc`, `good` o `vgood`.

Para leer los datos desde el fichero `cars.csv` que se proporciona se pueden evaluar las siguientes expresiones ([_Pandas_](http://pandas.pydata.org/) y [_NumPy_](http://www.numpy.org/) son paquetes de _Python_ para análisis de datos y cálculo científico, respectivamente):

In [1]:
import pandas
import numpy

cars = pandas.read_csv('cars.csv', header=None,
                       names=['buying', 'maint', 'doors', 'persons',
                              'lug_boot', 'safety', 'acceptability'])
print(cars.shape)  # Número de filas y columnas
cars.head(10)  # 10 primeras filas

(1728, 7)


,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


_sklearn_ no puede trabajar directamente con el conjunto de datos anterior, ya que asume que los valores de las variables discretas están codificadas con números enteros. Para transformar los datos a un formato adecuado ofrece diversas operaciones de preprocesamiento, entre las que se encuentran `OrdinalEncoder`, para codificar los atributos, y `LabelEncoder`, para codificar la variable objetivo.

In [2]:
from sklearn import preprocessing

atributos = cars.loc[:, 'buying':'safety']  # selección de las columnas de atributos
objetivo = cars['acceptability']  # selección de la columna objetivo

# Para realizar una codificación de los datos, se crea una instancia del tipo de codificación pretendida y se ajusta a los datos disponibles mediante el método
# fit. Los métodos transform e inverse_transform permiten entonces codificar y descodificar, respectivamente, los datos.

In [3]:
# El codificador adecuado para los atributos es OrdinalEncoder, ya que permite trabajar con el array completo de valores de los atributos.
codificador_atributos = preprocessing.OrdinalEncoder()
codificador_atributos.fit(atributos)
print(codificador_atributos.categories_)  # Categorías detectadas por el codificador para cada atributo
atributos_codificados = codificador_atributos.transform(atributos)
print(atributos_codificados)
print(codificador_atributos.inverse_transform([[3., 3., 0., 0., 2., 1.],
                                               [1., 1., 3., 2., 0., 1.]]))

[array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['2', '3', '4', '5more'], dtype=object), array(['2', '4', 'more'], dtype=object), array(['big', 'med', 'small'], dtype=object), array(['high', 'low', 'med'], dtype=object)]
[[3. 3. 0. 0. 2. 1.]
 [3. 3. 0. 0. 2. 2.]
 [3. 3. 0. 0. 2. 0.]
 ...
 [1. 1. 3. 2. 0. 1.]
 [1. 1. 3. 2. 0. 2.]
 [1. 1. 3. 2. 0. 0.]]
[['vhigh' 'vhigh' '2' '2' 'small' 'low']
 ['low' 'low' '5more' 'more' 'big' 'low']]


In [4]:
# El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja con la lista de sus valores, en lugar de con un array.
codificador_objetivo = preprocessing.LabelEncoder()
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)  # El método fit_transform ajusta la codificación y la aplica a los datos a continuación
print(codificador_objetivo.classes_)  # Clases detectadas por el codificador para la variable objetivo
print(objetivo_codificado)
print(codificador_objetivo.inverse_transform([2, 1, 3]))

['acc' 'good' 'unacc' 'vgood']
[2 2 2 ... 2 1 3]
['unacc' 'good' 'vgood']


Una vez codificadas las variables, es necesario separar el conjunto de datos en dos: un conjunto de entrenamiento, que se usará para generar los distintos modelos; y un conjunto de prueba, que se usará para comparar los distintos modelos.

Un detalle a tener en cuenta es que la distribución de ejemplos en las distintas clases de aceptabilidad no es uniforme: hay 1210 coches (un 70.023 % del total) clasificados como inaceptables (`unacc`), 384 coches (22.222 %) clasificados como aceptables (`acc`), 69 coches (3.993 %) clasificados como buenos (`good`) y 65 coches (3.762 %) clasificados como muy buenos (`vgood`).

Es conveniente, por tanto, que la separación de los ejemplos se realice de manera estratificada, es decir, intentando mantener la proporción anterior tanto en el conjunto de entrenamiento como en el de prueba.

Para dividir un conjunto de datos en un subconjunto de entrenamiento y otro de prueba, _sklearn_ proporciona la función `train_test_split`.

In [5]:
from sklearn import model_selection

print(cars.shape[0])  # Cantidad total de ejemplos
print(pandas.Series(objetivo).value_counts(normalize=True))  # Frecuencia total de cada clase de aceptabilidad

atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.33,  # Tamaño del conjunto de prueba
    stratify=objetivo_codificado)  # Estratificamos respecto a la distribución de valores en la variable objetivo

# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print(atributos_prueba.shape[0],
      len(objetivo_prueba),
      1728 * .33)
print(pandas.Series(codificador_objetivo.inverse_transform(objetivo_prueba)).value_counts(normalize=True))

# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print(atributos_entrenamiento.shape[0],
      len(objetivo_entrenamiento),
      1728 * .67)
print(pandas.Series(objetivo_entrenamiento).value_counts(normalize=True))

1728
unacc    0.700231
acc      0.222222
good     0.039931
vgood    0.037616
Name: acceptability, dtype: float64
571 571 570.24
unacc    0.700525
acc      0.222417
good     0.040280
vgood    0.036778
dtype: float64
1157 1157 1157.76
2    0.700086
0    0.222126
1    0.039758
3    0.038029
dtype: float64


Para realizar aprendizaje supervisado en _sklearn_ basta crear una instancia de la clase de objetos que implemente el modelo que se quiera utilizar (árboles de decisión, _naive_ Bayes, _kNN_, etc.).

Cada una de estas instancias dispondrá de los siguientes métodos:
* El método `fit` permite entrenar el modelo, dados __por separado__ el conjunto de ejemplos de entrenamiento y la clase de cada uno de estos ejemplos.
* El método `predict` permite clasificar un nuevo ejemplo una vez entrenado el modelo.
* El método `score` calcula el rendimiento del modelo, dados __por separado__ el conjunto de ejemplos de prueba y la clase de cada uno de estos ejemplos.

### Árboles de decisión

_sklearn_ implementa los árboles de decisión clasificadores como instancias de la clase `DecisionTreeClassifier`.

Desafortunadamente, son árboles de decisión binarios construidos asumiendo atributos continuos y mediante un algoritmo distinto a _ID3_, que no está implementado.

En http://scikit-learn.org/stable/modules/tree.html se puede encontrar información acerca de los árboles de decisión implementados en _sklearn_.

### _Naive_ Bayes

_sklearn_ implementa _naive_ Bayes para atributos categóricos mediante instancias de la clase `CategoricalNB`.

In [58]:
from sklearn import naive_bayes

clasif_NB = naive_bayes.CategoricalNB(alpha=1.0)  # alpha es el parámetro de suavizado
clasif_NB.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB()

Las siguientes expresiones muestran las cuentas realizadas y (los logaritmos de) las probabilidades aprendidas por el modelo.

In [7]:
print(clasif_NB.class_count_)
print(clasif_NB.class_log_prior_)
print(clasif_NB.category_count_)
print(clasif_NB.feature_log_prob_)

[257.  46. 810.  44.]
[-1.50450964 -3.22494433 -0.35655148 -3.26939609]
[array([[ 69.,  64.,  70.,  54.],
       [  0.,  29.,  17.,   0.],
       [230., 177., 174., 229.],
       [  0.,  26.,  18.,   0.]]), array([[ 62.,  65.,  78.,  52.],
       [  0.,  33.,  13.,   0.],
       [205., 186., 174., 245.],
       [ 12.,  17.,  15.,   0.]]), array([[ 54.,  60.,  77.,  66.],
       [ 11.,  13.,  13.,   9.],
       [218., 207., 199., 186.],
       [  7.,  11.,  11.,  15.]]), array([[  0., 131., 126.],
       [  0.,  23.,  23.],
       [377., 217., 216.],
       [  0.,  20.,  24.]]), array([[ 96.,  97.,  64.],
       [ 17.,  14.,  15.],
       [247., 273., 290.],
       [ 26.,  18.,   0.]]), array([[142.,   0., 115.],
       [ 22.,   0.,  24.],
       [178., 395., 237.],
       [ 44.,   0.,   0.]])]
[array([[-1.31602517, -1.39013314, -1.30184053, -1.55718722],
       [-3.91202301, -0.51082562, -1.02165125, -3.91202301],
       [-1.25954266, -1.52017682, -1.53717439, -1.26388106],
       [-3.

El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` la exactitud (_accuracy_) media sobre un conjunto de datos de prueba.

In [8]:
nuevos_ejemplos = [['vhigh', 'vhigh', '3', 'more', 'big', 'high'],
                   ['high', 'low', '3', '2', 'med', 'med']]
codificador_objetivo.inverse_transform(clasif_NB.predict(codificador_atributos.transform(nuevos_ejemplos)))

array(['unacc', 'unacc'], dtype=object)

In [9]:
# Calculamos la fracción de clases correctamente predichas para el conjunto de datos de prueba
clasif_NB.score(atributos_prueba, objetivo_prueba)

0.8441330998248686

### kNN

_sklearn_ implementa _kNN_ como instancias de la clase `KNeighborsClassifier`. En http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html se puede encontrar una descripción de las distancias actualmente implementadas que se podrían usar.

In [10]:
from sklearn import neighbors

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=5, metric='hamming')

Entrenamos el modelo.

In [11]:
clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='hamming')

El método `kneighbors` permite encontrar los (índices de los) $k$ vecinos más cercanos de los ejemplos proporcionados, así como las distancias a las que se encuentran.

In [12]:
distancias, vecinos = clasif_kNN.kneighbors(codificador_atributos.transform(nuevos_ejemplos))

# Vecinos más cercanos y distancia a ellos del primer ejemplo nuevo
print(nuevos_ejemplos[0])
print(codificador_atributos.inverse_transform(atributos_entrenamiento[vecinos[0]]))
print(distancias[0])
print(codificador_objetivo.inverse_transform(objetivo_entrenamiento[vecinos[0]]))

# Vecinos más cercanos y distancia a ellos del segundo ejemplo nuevo
print(nuevos_ejemplos[1])
print(codificador_atributos.inverse_transform(atributos_entrenamiento[vecinos[1]]))
print(distancias[1])
print(codificador_objetivo.inverse_transform(objetivo_entrenamiento[vecinos[1]]))

['vhigh', 'vhigh', '3', 'more', 'big', 'high']
[['high' 'vhigh' '3' 'more' 'big' 'high']
 ['vhigh' 'med' '3' 'more' 'big' 'high']
 ['med' 'vhigh' '3' 'more' 'big' 'high']
 ['low' 'vhigh' '3' 'more' 'big' 'high']
 ['vhigh' 'low' '3' 'more' 'big' 'high']]
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
['unacc' 'acc' 'acc' 'acc' 'acc']
['high', 'low', '3', '2', 'med', 'med']
[['high' 'low' '3' '2' 'med' 'med']
 ['high' 'low' '3' '2' 'big' 'med']
 ['high' 'low' '3' '2' 'med' 'low']
 ['high' 'low' '3' '2' 'med' 'high']
 ['low' 'low' '3' '2' 'med' 'med']]
[0.         0.16666667 0.16666667 0.16666667 0.16666667]
['unacc' 'unacc' 'unacc' 'unacc' 'unacc']


El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` la exactitud (_accuracy_) media sobre un conjunto de datos de prueba.

In [13]:
codificador_objetivo.inverse_transform(clasif_kNN.predict(codificador_atributos.transform(nuevos_ejemplos)))

array(['acc', 'unacc'], dtype=object)

In [14]:
clasif_kNN.score(atributos_prueba, objetivo_prueba)

0.8879159369527145

## Solicitudes de admisión en guarderías

El fichero de datos `nursery.csv` proporciona un conjunto de datos acerca de la evaluación de solicitudes de admisión en guarderías, en función de los siguientes atributos:
* _parents_, con posibles valores: `usual`, `pretentious`, `great_pret`.
* _has\_nurs_, con posibles valores: `proper`, `less_proper`, `improper`, `critical`, `very_crit`.
* _form_, con posibles valores: `complete`, `completed`, `incomplete`, `foster`.
* _children_, con posibles valores: `1`, `2`, `3`, `more`.
* _housing_, con posibles valores: `convenient`, `less_conv`, `critical`.
* _finance_, con posibles valores: `convenient`, `inconv`.
* _social_, con posibles valores: `non-prob`, `slightly_prob`, `problematic`.
* _health_, con posibles valores: `recommended`, `priority`, `not_recom`.

Los datos provienen de un sistema experto de decisión usado durante varios años de la década de los 80 en Liubliana (Eslovenia), que se desarrolló para poder proporcionar una explicación objetiva a las solicitudes rechazadas.

La evaluación de cada solicitud se indica mediante el atributo _evaluation_, que los clasifica como `not_recom`, `recommend`, `very_recom`, `priority` o `spec_prior`.

El objetivo es aprender a partir de los datos un modelo que prediga de la mejor forma posible cómo se evaluará una solicitud de admisión a partir de los valores de los atributos anteriores. Para ello se pide seguir los siguientes pasos:

* Leer los datos a partir del fichero `nursery.csv`.

In [2]:
import pandas
import numpy

nursery = pandas.read_csv('nursery.csv', header=None,
                       names=['parents', 'has_nurs', 'form', 'children',
                              'housing', 'finance', 'social', 'health', 'evaluation'])
print(nursery.shape)  # Número de filas y columnas
nursery.head(10)  # 10 primeras filas

(12958, 9)


,parents,has_nurs,form,children,housing,finance,social,health,evaluation
0,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
1,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
2,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
3,usual,proper,complete,1,convenient,convenient,slightly_prob,not_recom,not_recom
4,usual,proper,complete,1,convenient,convenient,problematic,recommended,priority
5,usual,proper,complete,1,convenient,convenient,problematic,priority,priority
6,usual,proper,complete,1,convenient,convenient,problematic,not_recom,not_recom
7,usual,proper,complete,1,convenient,inconv,nonprob,recommended,very_recom
8,usual,proper,complete,1,convenient,inconv,nonprob,priority,priority
9,usual,proper,complete,1,convenient,inconv,nonprob,not_recom,not_recom


* Codificar los datos con números enteros.

In [3]:
from sklearn import preprocessing

atributos2 = nursery.loc[:, 'parents':'health']  # selección de las columnas de atributos
objetivo2 = nursery['evaluation']  # selección de la columna objetivo

In [4]:
# El codificador adecuado para los atributos es OrdinalEncoder, ya que permite trabajar con el array completo de valores de los atributos.
codificador_atributos2 = preprocessing.OrdinalEncoder()
codificador_atributos2.fit(atributos2)
print(codificador_atributos2.categories_)  # Categorías detectadas por el codificador para cada atributo
atributos_codificados2 = codificador_atributos2.transform(atributos2)
print(atributos_codificados2)

[array(['great_pret', 'pretentious', 'usual'], dtype=object), array(['critical', 'improper', 'less_proper', 'proper', 'very_crit'],
      dtype=object), array(['complete', 'completed', 'foster', 'incomplete'], dtype=object), array(['1', '2', '3', 'more'], dtype=object), array(['convenient', 'critical', 'less_conv'], dtype=object), array(['convenient', 'inconv'], dtype=object), array(['nonprob', 'problematic', 'slightly_prob'], dtype=object), array(['not_recom', 'priority', 'recommended'], dtype=object)]
[[2. 3. 0. ... 0. 0. 1.]
 [2. 3. 0. ... 0. 0. 0.]
 [2. 3. 0. ... 0. 2. 1.]
 ...
 [0. 4. 2. ... 1. 1. 2.]
 [0. 4. 2. ... 1. 1. 1.]
 [0. 4. 2. ... 1. 1. 0.]]


In [5]:
# El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja con la lista de sus valores, en lugar de con un array.
codificador_objetivo2 = preprocessing.LabelEncoder()
objetivo_codificado2 = codificador_objetivo2.fit_transform(objetivo2)  # El método fit_transform ajusta la codificación y la aplica a los datos a continuación
print(codificador_objetivo2.classes_)  # Clases detectadas por el codificador para la variable objetivo
print(objetivo_codificado2)
print(codificador_objetivo2.inverse_transform([2, 1, 3]))

['not_recom' 'priority' 'spec_prior' 'very_recom']
[1 0 1 ... 2 2 0]
['spec_prior' 'priority' 'very_recom']


* Dividir el conjunto de datos en un subconjunto de entrenamiento (80 % de los datos) y un subconjunto de prueba (20 % de los datos). El primero de ellos se utilizará tanto para seleccionar un tipo de modelo utilizando la técnica de validación cruzada, como para entrenar el modelo finalmente seleccionado. El segundo se utilizará para medir la capacidad de generalización de este último.

In [6]:
from sklearn import model_selection

print(nursery.shape[0])  # Cantidad total de ejemplos
print(pandas.Series(objetivo2).value_counts(normalize=True))  # Frecuencia total de cada clase de aceptabilidad

atributos_entrenamiento2, atributos_prueba2, objetivo_entrenamiento2, objetivo_prueba2 = model_selection.train_test_split(
    atributos_codificados2, objetivo_codificado2,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.20,  # Tamaño del conjunto de prueba
    stratify=objetivo_codificado2)  # Estratificamos respecto a la distribución de valores en la variable objetivo

# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print(atributos_prueba2.shape[0],
      len(objetivo_prueba2),
      1728 * .20)
print(pandas.Series(codificador_objetivo2.inverse_transform(objetivo_prueba2)).value_counts(normalize=True))

# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print(atributos_entrenamiento2.shape[0],
      len(objetivo_entrenamiento2),
      1728 * .80)
print(pandas.Series(objetivo_entrenamiento2).value_counts(normalize=True))

12958
not_recom     0.333385
priority      0.329217
spec_prior    0.312085
very_recom    0.025313
Name: evaluation, dtype: float64
2592 2592 345.6
not_recom     0.333333
priority      0.329090
spec_prior    0.312114
very_recom    0.025463
dtype: float64
10366 10366 1382.4
0    0.333398
1    0.329249
2    0.312078
3    0.025275
dtype: float64


* Usando el subconjunto de entrenamiento y utilizando la técnica de validación cruzada con 10 particiones, estimar la exactitud (_accuracy_) media de un modelo de tipo _naive_ Bayes con suavizado ɑ, para cada uno de los valores ɑ = 1, ..., 10, y de un modelo de tipo kNN, para cada uno de los valores k = 1, ..., 10.

__Nota__: la función `cross_val_score` del módulo `model_selection` de _sklearn_ implementa el procedimiento de validación cruzada. Admite, entre otros, los siguientes argumentos:
* _estimator_: modelo a evaluar.
* _X_: array con los valores de los atributos de los ejemplos.
* _y_: array con las clasificaciones de los ejemplos.
* _cv_: número de subconjuntos en los que dividir los datos.

Devuelve un array con el rendimiento del modelo sobre cada subconjunto, una vez entrenado con los ejemplos del resto de subconjuntos.

Para más información acerca de la implementación en _sklearn_ del método de validación cruzada puede consultarse http://scikit-learn.org/stable/modules/cross_validation.html.

In [7]:
nuevos_ejemplos2 = [['usual', 'proper','completed','2', 'convenient', 'inconv', 'slightly_prob','recommended'],
                   ['usual', 'proper', 'complete','1', 'critical', 'convenient', 'nonprob','priority']]

In [19]:
from sklearn import naive_bayes

for a in range(1,10):
    clasif_NB = naive_bayes.CategoricalNB(alpha=a)  # alpha es el parámetro de suavizado
    clasif_NB.fit(atributos_entrenamiento2, objetivo_entrenamiento2)
    codificador_objetivo2.inverse_transform(clasif_NB.predict(codificador_atributos2.transform(nuevos_ejemplos2)))
    clasif_NB.score(atributos_prueba2, objetivo_prueba2)
    rend = model_selection.cross_val_score(
        clasif_NB.fit(atributos_entrenamiento2, objetivo_entrenamiento2),
        atributos_codificados2,
        objetivo_codificado2, 
        cv=10)
    print(rend)


[0.82716049 0.91820988 0.81481481 0.8287037  0.8433642  0.66358025
 0.78240741 0.80555556 0.93513514 0.8007722 ]
[0.82561728 0.91743827 0.81481481 0.8287037  0.8433642  0.66358025
 0.78317901 0.80555556 0.93513514 0.8007722 ]
[0.82098765 0.91666667 0.81481481 0.8287037  0.8433642  0.66358025
 0.78395062 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91666667 0.8132716  0.8287037  0.8433642  0.66358025
 0.78395062 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91512346 0.8132716  0.8287037  0.8433642  0.66358025
 0.78395062 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91512346 0.8132716  0.8287037  0.8433642  0.66358025
 0.78395062 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91512346 0.8132716  0.8287037  0.8433642  0.66358025
 0.78549383 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91512346 0.8132716  0.8287037  0.8441358  0.66358025
 0.78549383 0.80709877 0.93513514 0.8007722 ]
[0.81944444 0.91512346 0.8132716  0.8287037  0.84490741 0.66358025
 0.78626543 0.80709877 0.9351

In [18]:
rend.score(atributos_prueba2, objetivo_prueba2)

AttributeError: 'numpy.ndarray' object has no attribute 'score'

In [11]:
rend = model_selection.cross_val_score(
    clasif_NB.fit(atributos_entrenamiento2, objetivo_entrenamiento2),
    atributos_codificados2,
    objetivo_codificado2, 
    cv=10)
print(rend)

[0.81944444 0.91512346 0.8132716  0.8287037  0.84490741 0.66358025
 0.78626543 0.80709877 0.93513514 0.8007722 ]


* Con el mejor valor de suavizado determinado anteriormente, entrenar el algoritmo _naive_ Bayes con el subconjunto de entrenamiento y calcular la exactitud media sobre el subconjunto de prueba.

* Con el mejor número de vecinos determinado anteriormente, entrenar el algoritmo _kNN_ con el subconjunto de entrenamiento y calcular la exactitud media sobre el subconjunto de prueba.

In [14]:
from sklearn import neighbors

clasif_kNN2 = neighbors.KNeighborsClassifier(n_neighbors=10, metric='hamming')

In [16]:
clasif_kNN2.fit(atributos_entrenamiento2, objetivo_entrenamiento2)

KNeighborsClassifier(metric='hamming', n_neighbors=10)

In [17]:
distancias2, vecinos2 = clasif_kNN2.kneighbors(codificador_atributos2.transform(nuevos_ejemplos2))

# Vecinos más cercanos y distancia a ellos del primer ejemplo nuevo
print(nuevos_ejemplos2[0])
print(codificador_atributos2.inverse_transform(atributos_entrenamiento2[vecinos2[0]]))
print(distancias2[0])
print(codificador_objetivo2.inverse_transform(objetivo_entrenamiento2[vecinos2[0]]))

# Vecinos más cercanos y distancia a ellos del segundo ejemplo nuevo
print(nuevos_ejemplos2[1])
print(codificador_atributos2.inverse_transform(atributos_entrenamiento2[vecinos2[1]]))
print(distancias2[1])
print(codificador_objetivo2.inverse_transform(objetivo_entrenamiento2[vecinos2[1]]))

['usual', 'proper', 'completed', '2', 'convenient', 'inconv', 'slightly_prob', 'recommended']
[['usual' 'critical' 'completed' '2' 'convenient' 'inconv'
  'slightly_prob' 'recommended']
 ['usual' 'proper' 'completed' '2' 'convenient' 'inconv' 'problematic'
  'recommended']
 ['usual' 'less_proper' 'completed' '2' 'convenient' 'inconv'
  'slightly_prob' 'recommended']
 ['usual' 'proper' 'complete' '2' 'convenient' 'inconv' 'slightly_prob'
  'recommended']
 ['usual' 'proper' 'completed' '2' 'convenient' 'inconv' 'nonprob'
  'recommended']
 ['pretentious' 'proper' 'completed' '2' 'convenient' 'inconv'
  'slightly_prob' 'recommended']
 ['usual' 'proper' 'completed' '2' 'convenient' 'inconv' 'slightly_prob'
  'not_recom']
 ['great_pret' 'proper' 'completed' '2' 'convenient' 'inconv'
  'slightly_prob' 'recommended']
 ['usual' 'proper' 'completed' '1' 'convenient' 'inconv' 'slightly_prob'
  'recommended']
 ['usual' 'proper' 'completed' '2' 'less_conv' 'inconv' 'slightly_prob'
  'recommended']]

* ¿Cuál de los dos modelos construidos en los puntos anteriores realiza mejores predicciones acerca de la evaluación de las solicitudes de admisión?